## **SVR**

In [1]:
# !gdown --id '1zpj8i4Lx27hzX7Go86IdZ0d0exCXKoDe&export' --output nba2122.zip
# !gdown --id '1_16P0k9O5i8mYOIw9oWPzHqXQ2PMMZWn&export' --output team.p
!gdown --id '1LE_WhdXwi2dgsbDdvUJ83VfLXpyttYoO&export' --output nba2223.zip
!gdown --id '1_16P0k9O5i8mYOIw9oWPzHqXQ2PMMZWn&export' --output team.p

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1LE_WhdXwi2dgsbDdvUJ83VfLXpyttYoO&export
To: /content/nba2223.zip
100% 7.40M/7.40M [00:00<00:00, 56.8MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_16P0k9O5i8mYOIw9oWPzHqXQ2PMMZWn&export
To: /content/team.p
100% 346/346 [00:00<00:00, 2.01MB/s]


In [2]:
# !unzip -o nba2122.zip
!unzip -o nba2223.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: def/01-17-2023/1610612749.csv  
  inflating: def/01-17-2023/1610612751.csv  
  inflating: def/01-17-2023/1610612755.csv  
  inflating: def/01-17-2023/1610612757.csv  
  inflating: def/01-17-2023/1610612759.csv  
  inflating: def/01-17-2023/1610612761.csv  
   creating: def/01-18-2023/
  inflating: def/01-18-2023/1610612737.csv  
  inflating: def/01-18-2023/1610612739.csv  
  inflating: def/01-18-2023/1610612740.csv  
  inflating: def/01-18-2023/1610612742.csv  
  inflating: def/01-18-2023/1610612743.csv  
  inflating: def/01-18-2023/1610612745.csv  
  inflating: def/01-18-2023/1610612746.csv  
  inflating: def/01-18-2023/1610612747.csv  
  inflating: def/01-18-2023/1610612748.csv  
  inflating: def/01-18-2023/1610612750.csv  
  inflating: def/01-18-2023/1610612752.csv  
  inflating: def/01-18-2023/1610612754.csv  
  inflating: def/01-18-2023/1610612758.csv  
  inflating: def/01-18-2023/1610612760.csv  
  inflating: def/01-18-2023/1610612762.csv  
  inf

In [3]:
import numpy as np
import pandas as pd
import os
import pickle
from sklearn import svm
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [4]:
def calculate_averages(list_2d):
  column_totals = dict()
  for row_idx, row in enumerate(list_2d):
    for cell_idx, cell in enumerate(row):
      if type(cell) in [int, float, complex]:
        if cell_idx in column_totals:
          column_totals[cell_idx].append(cell)
        else:
          column_totals[cell_idx] = [cell]
  per_col_avg = [sum(column_totals[col_idx]) / len(column_totals[col_idx]) for col_idx in column_totals]
  return per_col_avg

## LR

In [7]:
# schedule = pickle.load(open("schedule3.p", "rb"))
# y = pickle.load(open("team_list_y_namenum_ave_odds.p", "rb"))
# tean_dict = pickle.load(open("team.p", "rb"))
# start_date = '10/19/2021'
# end_date = '04/15/2022'
schedule = pickle.load(open("schedule4.p", "rb"))
y = pickle.load(open("team_list_y_namenum_ave_odds.p", "rb"))
tean_dict = pickle.load(open("team.p", "rb"))
start_date = '10/18/2022'
end_date = '04/09/2023'
team_dict = dict(zip(range(30), tean_dict.values()))
date_list = [d.strftime('%m-%d-%Y') for d in pd.date_range(start=start_date, end=end_date)]
assert len(y) == len(date_list)
data_len = len(date_list)
split_ratio = 0.8
train_size = int(data_len * split_ratio)
train_x = []
train_y = []
test_x = []
test_y = []
test_y_odds = []
seq_len = 5
pre_len = 1
feature_list = ['W', 'L', 'PTS', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'AST', 'TOV', 'STL', 'BLK', 'PF']

In [8]:
team_record_dict = dict.fromkeys(tean_dict.values())
for k in team_record_dict.keys():
  team_record_dict[k] = list()

day_count = 0
for day in date_list:
  match_list = schedule[day.replace("-", "/")]
  for _ in match_list:
    team_record_dict[tean_dict[list(_.keys())[0]]].append(day_count)
    team_record_dict[tean_dict[list(_.keys())[1]]].append(day_count)
  day_count += 1

In [9]:
x_dict = dict.fromkeys(range(30))
for k in x_dict.keys():
  x_dict[k] = list()
for i in range(seq_len):
  for match_ in y[i].keys():
    df_t1 = pd.read_csv(os.path.join('team', date_list[i], team_dict[match_[0]]+'.csv'))
    df_t1 = df_t1[feature_list]
    x_dict[match_[0]].append(df_t1.values.tolist()[0])
    df_t2 = pd.read_csv(os.path.join('team', date_list[i], team_dict[match_[1]]+'.csv'))
    df_t2 = df_t2[feature_list]
    x_dict[match_[1]].append(df_t2.values.tolist()[0])

for i in range(data_len - seq_len - pre_len):
  for match_ in y[i+seq_len].keys():
    team1_ave = x_dict[match_[0]]
    team2_ave = x_dict[match_[1]]
    if len(team1_ave) < seq_len or len(team2_ave) < seq_len:
      continue
    team1_s_ave = calculate_averages(team1_ave)
    team2_s_ave = calculate_averages(team2_ave)
    res_x = []
    res_x.extend(team1_s_ave)
    res_x.extend(team2_s_ave)
    if i + seq_len > train_size:
      test_x.append(res_x)
      test_y_odds.append((y[i+seq_len][match_][5], y[i+seq_len][match_][6]))
      test_y.append(y[i+seq_len][match_][0])
    else:
      train_x.append(res_x)
      train_y.append(y[i+seq_len][match_][0])

  for match_ in y[i+seq_len].keys():
    df_t1 = pd.read_csv(os.path.join('team', date_list[i+seq_len], team_dict[match_[0]]+'.csv'))
    df_t1 = df_t1[feature_list]
    x_dict[match_[0]].append(df_t1.values.tolist()[0])
    # t1_odds = y[i][match_][5]
    df_t2 = pd.read_csv(os.path.join('team', date_list[i+seq_len], team_dict[match_[1]]+'.csv'))
    df_t2 = df_t2[feature_list]
    x_dict[match_[1]].append(df_t2.values.tolist()[0])
    # t2_odds = y[i][match_][6]

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
model = LinearRegression()
model.fit(train_x, train_y)
predicted = model.predict(test_x)

In [12]:
mean_squared_error(predicted, test_y, squared=False)

13.816569894794345

In [13]:
mean_absolute_error(predicted, test_y)

11.236475185466452

In [14]:
game = 0.0
correct = 0.0
gain = 0.0
for i in range(len(predicted)):
  game += 1
  if predicted[i] * test_y[i] > 0:
    correct += 1
    if predicted[i] > 0:
      gain += test_y_odds[i][0]
    else:
      gain += test_y_odds[i][1]
print(correct / game)

0.5991561181434599


In [ ]:
print(gain / game)

0.9829074889867842


## SVR

In [15]:
# schedule = pickle.load(open("schedule3.p", "rb"))
# y = pickle.load(open("team_list_y_namenum_ave_odds.p", "rb"))
# tean_dict = pickle.load(open("team.p", "rb"))
# start_date = '10/19/2021'
# end_date = '04/15/2022'
schedule = pickle.load(open("schedule4.p", "rb"))
y = pickle.load(open("team_list_y_namenum_ave_odds.p", "rb"))
tean_dict = pickle.load(open("team.p", "rb"))
start_date = '10/18/2022'
end_date = '04/09/2023'
team_dict = dict(zip(range(30), tean_dict.values()))
date_list = [d.strftime('%m-%d-%Y') for d in pd.date_range(start=start_date, end=end_date)]
assert len(y) == len(date_list)
data_len = len(date_list)
split_ratio = 0.8
train_size = int(data_len * split_ratio)
train_x = []
train_y = []
test_x = []
test_y = []
test_y_odds = []
seq_len = 5
pre_len = 1
feature_list = ['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS']

In [16]:
team_record_dict = dict.fromkeys(tean_dict.values())
for k in team_record_dict.keys():
  team_record_dict[k] = list()

day_count = 0
for day in date_list:
  match_list = schedule[day.replace("-", "/")]
  for _ in match_list:
    team_record_dict[tean_dict[list(_.keys())[0]]].append(day_count)
    team_record_dict[tean_dict[list(_.keys())[1]]].append(day_count)
  day_count += 1

In [17]:
x_dict = dict.fromkeys(range(30))
for k in x_dict.keys():
  x_dict[k] = list()
for i in range(seq_len):
  for match_ in y[i].keys():
    df_t1 = pd.read_csv(os.path.join('team', date_list[i], team_dict[match_[0]]+'.csv'))
    df_t1 = df_t1[feature_list]
    x_dict[match_[0]].append(df_t1.values.tolist()[0])
    df_t2 = pd.read_csv(os.path.join('team', date_list[i], team_dict[match_[1]]+'.csv'))
    df_t2 = df_t2[feature_list]
    x_dict[match_[1]].append(df_t2.values.tolist()[0])

for i in range(data_len - seq_len - pre_len):
  for match_ in y[i+seq_len].keys():
    team1_ave = x_dict[match_[0]]
    team2_ave = x_dict[match_[1]]
    if len(team1_ave) < 5 or len(team2_ave) < 5:
      continue
    team1_s_ave = calculate_averages(team1_ave)
    team1_s_ave.extend(calculate_averages(team1_ave[-5:]))
    team1_s_ave.extend(team1_ave[-5])
    team1_s_ave.extend(team1_ave[-4])
    team1_s_ave.extend(team1_ave[-3])
    team1_s_ave.extend(team1_ave[-2])
    team1_s_ave.extend(team1_ave[-1])
    team2_s_ave = calculate_averages(team2_ave)
    team2_s_ave.extend(calculate_averages(team2_ave[-5:]))
    team2_s_ave.extend(team2_ave[-5])
    team2_s_ave.extend(team2_ave[-4])
    team2_s_ave.extend(team2_ave[-3])
    team2_s_ave.extend(team2_ave[-2])
    team2_s_ave.extend(team2_ave[-1])
    res_x = []
    res_x.extend(team1_s_ave)
    res_x.extend(team2_s_ave)
    rest_1 = list()
    rest_2 = list()
    for cool_seq in range(5):
      if i+seq_len-1-cool_seq in team_record_dict[team_dict[match_[0]]]:
        rest_1.append(1)
      else:
        rest_1.append(0)
      if i+seq_len-1-cool_seq in team_record_dict[team_dict[match_[1]]]:
        rest_2.append(1)
      else:
        rest_2.append(0)
    res_x.extend(rest_1)
    res_x.extend(rest_2)
    if i + seq_len > train_size:
      test_x.append(res_x)
      test_y_odds.append((y[i+seq_len][match_][5], y[i+seq_len][match_][6]))
      test_y.append(y[i+seq_len][match_][0])
    else:
      train_x.append(res_x)
      train_y.append(y[i+seq_len][match_][0])

  for match_ in y[i+seq_len].keys():
    df_t1 = pd.read_csv(os.path.join('team', date_list[i+seq_len], team_dict[match_[0]]+'.csv'))
    df_t1 = df_t1[feature_list]
    x_dict[match_[0]].append(df_t1.values.tolist()[0])
    # t1_odds = y[i][match_][5]
    df_t2 = pd.read_csv(os.path.join('team', date_list[i+seq_len], team_dict[match_[1]]+'.csv'))
    df_t2 = df_t2[feature_list]
    x_dict[match_[1]].append(df_t2.values.tolist()[0])
    # t2_odds = y[i][match_][6]

In [18]:
model = svm.SVR(kernel ='rbf', degree=3, gamma='auto', C=1)
model.fit(train_x, train_y)
predicted = model.predict(test_x)

In [19]:
mean_squared_error(predicted, test_y, squared=False)

14.933576901327067

In [20]:
mean_absolute_error(predicted, test_y)

11.886179444307537

In [21]:
game = 0.0
correct = 0.0
gain = 0.0
for i in range(len(predicted)):
  game += 1
  if predicted[i] * test_y[i] > 0:
    correct += 1
    if predicted[i] > 0:
      gain += test_y_odds[i][0]
    else:
      gain += test_y_odds[i][1]
print(correct / game)

0.5443037974683544


In [ ]:
print(gain / game)

0.9286784140969164


# RF

In [ ]:
!gdown --id '1zpj8i4Lx27hzX7Go86IdZ0d0exCXKoDe&export' --output nba2122.zip
!gdown --id '1_16P0k9O5i8mYOIw9oWPzHqXQ2PMMZWn&export' --output team.p
!unzip -o nba2122.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: pass/01-17-2022/NYK.csv  
  inflating: pass/01-17-2022/OKC.csv  
  inflating: pass/01-17-2022/ORL.csv  
  inflating: pass/01-17-2022/PHI.csv  
  inflating: pass/01-17-2022/PHX.csv  
  inflating: pass/01-17-2022/POR.csv  
  inflating: pass/01-17-2022/SAS.csv  
  inflating: pass/01-17-2022/TOR.csv  
  inflating: pass/01-17-2022/UTA.csv  
  inflating: pass/01-17-2022/WAS.csv  
   creating: pass/01-18-2022/
  inflating: pass/01-18-2022/DET.csv  
  inflating: pass/01-18-2022/GSW.csv  
  inflating: pass/01-18-2022/MIN.csv  
  inflating: pass/01-18-2022/NYK.csv  
   creating: pass/01-19-2022/
  inflating: pass/01-19-2022/ATL.csv  
  inflating: pass/01-19-2022/BKN.csv  
  inflating: pass/01-19-2022/BOS.csv  
  inflating: pass/01-19-2022/CHA.csv  
  inflating: pass/01-19-2022/CHI.csv  
  inflating: pass/01-19-2022/CLE.csv  
  inflating: pass/01-19-2022/DAL.csv  
  inflating: pass/01-19-2022/DEN.csv  
  inflating: pass/01-19-2022/DET.csv  
  inflating: pass/01-1

In [22]:
import numpy as np
import pandas as pd
import math
import os
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [23]:
# start_date = '10/19/2021'
# end_date = '04/15/2022'
start_date = '10/18/2022'
end_date = '04/09/2023'
date_list = [d.strftime('%m-%d-%Y') for d in pd.date_range(start=start_date, end=end_date)]
data_len = len(date_list)
split_ratio = 0.8
train_size = int(data_len * split_ratio)
for i in range(len(date_list)):
  if i == train_size:
    print(date_list[i])

03-06-2023


In [24]:
# schedule = pickle.load(open("schedule3.p", "rb"))
# y = pickle.load(open("team_list_y_namenum_ave_odds.p", "rb"))
# tean_dict = pickle.load(open("team.p", "rb"))
# start_date = '10/19/2021'
# end_date = '04/15/2022'
schedule = pickle.load(open("schedule4.p", "rb"))
y = pickle.load(open("team_list_y_namenum_ave_odds.p", "rb"))
tean_dict = pickle.load(open("team.p", "rb"))
start_date = '10/18/2022'
end_date = '04/09/2023'
team_dict = dict(zip(range(30), tean_dict.values()))
date_list = [d.strftime('%m-%d-%Y') for d in pd.date_range(start=start_date, end=end_date)]
assert len(y) == len(date_list)
data_len = len(date_list)
split_ratio = 0.8
train_size = int(data_len * split_ratio)
train_x = []
train_y = []
test_x = []
test_y = []
test_y_odds = []
seq_len = 5
pre_len = 1
feature_list = ['PTS', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA']

In [25]:
def winning_probability(eloA, eloB, hca=69):
  return 1 / (1 + math.pow(10, (-(eloA - eloB + hca) / 400)))

def get_k(eloA, eloB, mov):
  return 20 * math.pow((mov + 3), 0.8) / (7.5 + (0.006 * abs(eloA - eloB)))

def del_elo(eloA, eloB, mov):
  return get_k(eloA, eloB, mov) * (1 - winning_probability(eloA, eloB))

In [26]:
score_list = [1500] * 30
elo_dict = dict(zip(range(30), score_list))
elo_list = []
for element in y:
  day_elo_dict = dict()
  for match_ in element.keys():
    day_elo_dict[match_] = (elo_dict[match_[0]], elo_dict[match_[1]])
    if element[match_][0] > 0:
      score = del_elo(elo_dict[match_[0]], elo_dict[match_[1]], element[match_][0])
      elo_dict[match_[0]] += score
      elo_dict[match_[1]] -= score
    else:
      score = del_elo(elo_dict[match_[1]], elo_dict[match_[0]], -element[match_][0])
      elo_dict[match_[1]] += score
      elo_dict[match_[0]] -= score
  elo_list.append(day_elo_dict)

In [27]:
def calculate_averages(list_2d):
  column_totals = dict()
  for row_idx, row in enumerate(list_2d):
    for cell_idx, cell in enumerate(row):
      if type(cell) in [int, float, complex]:
        if cell_idx in column_totals:
          column_totals[cell_idx].append(cell)
        else:
          column_totals[cell_idx] = [cell]
  per_col_avg = [sum(column_totals[col_idx]) / len(column_totals[col_idx]) for col_idx in column_totals]
  return per_col_avg

In [28]:
team_record_dict = dict.fromkeys(tean_dict.values())
for k in team_record_dict.keys():
  team_record_dict[k] = dict()

day_count = 0
for day in date_list:
  match_list = schedule[day.replace("-", "/")]
  for _ in match_list:
    team_record_dict[tean_dict[list(_.keys())[0]]][day_count] = 0
    team_record_dict[tean_dict[list(_.keys())[1]]][day_count] = 1
  day_count += 1

In [29]:
x_dict = dict.fromkeys(range(30))
for k in x_dict.keys():
  x_dict[k] = list()
for i in range(seq_len):
  for match_ in y[i].keys():
    df_t1 = pd.read_csv(os.path.join('team', date_list[i], team_dict[match_[0]]+'.csv'))
    df_t1 = df_t1[feature_list]
    to_append_list = df_t1.values.tolist()[0]
    to_append_list.append(elo_list[i][match_][0])
    x_dict[match_[0]].append(to_append_list)
    df_t2 = pd.read_csv(os.path.join('team', date_list[i], team_dict[match_[1]]+'.csv'))
    df_t2 = df_t2[feature_list]
    to_append_list = df_t2.values.tolist()[0]
    to_append_list.append(elo_list[i][match_][1])
    x_dict[match_[1]].append(to_append_list)

for i in range(data_len - seq_len - pre_len):
  for match_ in y[i+seq_len].keys():
    team1_ave = x_dict[match_[0]]
    team2_ave = x_dict[match_[1]]
    if len(team1_ave) < 5 or len(team2_ave) < 5:
      continue
    team1_s_ave = calculate_averages(team1_ave[-10:])
    team2_s_ave = calculate_averages(team2_ave[-10:])
    res_x = []
    res_x.extend(team1_s_ave)
    res_x.extend(team2_s_ave)
    rest_1 = list()
    rest_2 = list()
    for cool_seq in range(2):
      if i+seq_len-1-cool_seq in team_record_dict[team_dict[match_[0]]].keys():
        rest_1.append(team_record_dict[team_dict[match_[0]]][i+seq_len-1-cool_seq])
      else:
        rest_1.append(-1)
      if i+seq_len-1-cool_seq in team_record_dict[team_dict[match_[1]]].keys():
        rest_2.append(team_record_dict[team_dict[match_[1]]][i+seq_len-1-cool_seq])
      else:
        rest_2.append(-1)
    if rest_1[0] == -1 and rest_1[1] == -1:
      res_x.append(0)
    elif rest_1[1] != -1:
      res_x.append(3)
    elif rest_1[0] == 0:
      res_x.append(1)
    else:
      res_x.append(2)
    if rest_2[0] == -1 and rest_2[1] == -1:
      res_x.append(0)
    elif rest_2[1] != -1:
      res_x.append(3)
    elif rest_2[0] == 1:
      res_x.append(3)
    else:
      res_x.append(2)
    if i + seq_len > train_size:
      test_x.append(res_x)
      test_y_odds.append((y[i+seq_len][match_][5], y[i+seq_len][match_][6]))
      test_y.append(y[i+seq_len][match_][0])
    else:
      train_x.append(res_x)
      train_y.append(y[i+seq_len][match_][0])

  for match_ in y[i+seq_len].keys():
    df_t1 = pd.read_csv(os.path.join('team', date_list[i+seq_len], team_dict[match_[0]]+'.csv'))
    df_t1 = df_t1[feature_list]
    to_append_list = df_t1.values.tolist()[0]
    to_append_list.append(elo_list[i+seq_len][match_][0])
    x_dict[match_[0]].append(to_append_list)
    df_t2 = pd.read_csv(os.path.join('team', date_list[i+seq_len], team_dict[match_[1]]+'.csv'))
    df_t2 = df_t2[feature_list]
    x_dict[match_[1]].append(df_t2.values.tolist()[0])
    to_append_list = df_t2.values.tolist()[0]
    to_append_list.append(elo_list[i+seq_len][match_][1])
    x_dict[match_[1]].append(to_append_list)

In [30]:
regr = RandomForestRegressor(random_state=0)
regr.fit(train_x, train_y)
predicted = regr.predict(test_x)

In [31]:
mean_squared_error(predicted, test_y, squared=False)

13.69849192475793

In [32]:
mean_absolute_error(predicted, test_y)

10.986751054852324

In [33]:
game = 0.0
correct = 0.0
gain = 0.0
for i in range(len(predicted)):
  game += 1
  if predicted[i] * test_y[i] > 0:
    correct += 1
    if predicted[i] > 0:
      gain += test_y_odds[i][0]
    else:
      gain += test_y_odds[i][1]
print(correct / game)

0.6118143459915611


In [ ]:
print(gain / game)

0.973832599118943
